## MNIST Distributed Training

### Overview
Distributed training of a CNN model on MNIST handwritten digit dataset using PyTorch's DistributedDataParallel (DDP).
### Training Setup
- **Backend**: NCCL (GPU) / Gloo (CPU)
- **Data**: MNIST test set (train=False)
- **Distributed**: Multi-Node Multi-CPU/GPU with DistributedSampler
- **Device Flexibility**: Auto-fallback GPU→CPU
### Key Parameters
- `epochs`: Training iterations
- `batch_size`: Samples per device
- `lr`: Learning rate
- `save_every`: Checkpoint frequency
- `backend`: Communication backend

In [ ]:
%pip install dist/kubeflow-0.2.0.dev0-py3-none-any.whl --force-reinstall

In [1]:
%pip show kubeflow

Name: kubeflow
Version: 0.1.0
Summary: Kubeflow Python SDK to manage ML workloads and to interact with Kubeflow APIs.
Home-page: https://github.com/kubeflow/sdk
Author: 
Author-email: The Kubeflow Authors <kubeflow-discuss@googlegroups.com>
License: 
Location: /opt/app-root/lib64/python3.12/site-packages
Requires: kubeflow-trainer-api, kubernetes, pydantic
Required-by: 
Note: you may need to restart the kernel to use updated packages.


### Initialize Training Client

This section demonstrates the new SDK import structure and client initialization.

In [ ]:
from kubeflow.trainer import TrainerClient
from kubeflow.trainer.backends.kubernetes.types import KubernetesBackendConfig
from kubernetes import client

# Configure Kubernetes client
# Replace with your cluster details
api_server = ""
token = ""
namespace = "test-ns"

configuration = client.Configuration()
configuration.host = api_server
configuration.api_key = {"authorization": f"Bearer {token}"}

# Uncomment if your cluster API server uses a self-signed certificate
# configuration.verify_ssl = False

api_client = client.ApiClient(configuration)
trainer_client = TrainerClient(
    backend_config=KubernetesBackendConfig(
        namespace=namespace, 
        client_configuration=api_client.configuration
    )
)

print("✓ Trainer client initialized")
print(f"Available runtimes: {len(trainer_client.list_runtimes())}")
for runtime in trainer_client.list_runtimes():
    print(f"  🔧 {runtime.name}: {runtime.trainer.framework} ({runtime.trainer.num_nodes} nodes)")

✓ Trainer client initialized
Available runtimes: 4
  - torch-cuda-241: torch (1 nodes)
  - torch-cuda-251: torch (1 nodes)
  - torch-rocm-241: torch (1 nodes)
  - torch-rocm-251: torch (1 nodes)


### Submit Training Jobs - Demonstrating New SDK Features

This section showcases the new SDK features:
1. **Backend-specific imports** for better organization
2. **Option 2 features** for container customization
3. **Enhanced options** with override behavior
4. **Multiple training scenarios** (traditional, hybrid, container-only)

In [ ]:
from kubeflow.trainer import CustomTrainer, WithTrainerImage, WithTrainerCommand, WithTrainerArgs

# Backend-specific imports
from kubeflow.trainer.backends.kubernetes.options import (
    WithLabels, WithAnnotations, WithName, WithPodSpecOverrides, PodSpecOverride
)

# Import the training function
from kfto_mnist import main

# Traditional approach with enhanced options
traditional_trainer = CustomTrainer(
    func=main,
    func_args={
        "epochs": 3,
        "save_every": 1, 
        "batch_size": 4, 
        "backend": "gloo",  # CPU backend for demo
        "lr": 0.001, 
        "dataset_path": "/shared/data", 
        'snapshot_path': "/shared/checkpoints/snapshot_mnist.pt"
    },
    num_nodes=2,
    resources_per_node={
        "cpu": "2",
        "memory": "4Gi",
    },
    packages_to_install=["torchvision"]
)

# Enhanced options with override behavior
traditional_options = [
    WithName("mnist-traditional-demo"),
    WithLabels({
        "team": "ml-platform",
        "project": "training-experiment",
        "approach": "traditional",
        "sdk-version": "0.2.0.dev0"
    }),
    WithAnnotations({
        "created-by": "kubeflow-sdk-v2",
        "training-type": "function-based"
    }),
    WithPodSpecOverrides([
        PodSpecOverride(
            target_jobs=["node"],
            volumes=[{
                "name": "shared",
                "persistentVolumeClaim": {"claimName": "shared"}
            }],
            containers=[{
                "name": "node",
                "volumeMounts": [{"name": "shared", "mountPath": "/shared"}],
                "env": [
                    {"name": "TRAINING_MODE", "value": "traditional"},
                    {"name": "SDK_VERSION", "value": "0.2.0.dev0"}
                ]
            }]
        )
    ])
]

# Submit traditional training job
job1_name = trainer_client.train(
    trainer=traditional_trainer,
    runtime=trainer_client.get_runtime("torch-cuda-251"),
    options=traditional_options
)

print(f" Traditional TrainJob '{job1_name}' submitted!")

print("\n DEMO 2: Hybrid Training (Function + Container Customization)")
print("=" * 60)

# Hybrid approach: Function + Option 2 container customization
hybrid_trainer = CustomTrainer(
    func=main,
    func_args={
        "epochs": 2,  # Quick demo
        "save_every": 1,
        "batch_size": 4,
        "backend": "gloo",
        "lr": 0.001,
        "dataset_path": "/tmp/data",  # Use tmp for demo
        "snapshot_path": "/tmp/snapshot_mnist.pt"
    },
    num_nodes=1,
    resources_per_node={
        "cpu": "2",
        "memory": "4Gi"
    },
    packages_to_install=["torchvision"]
)

# Hybrid options: Traditional + Option 2 features
hybrid_options = [
    WithName("mnist-hybrid-demo"),
    WithLabels({
        "approach": "hybrid",
        "has-function": "true",
        "has-container-customization": "true"
    }),
    WithAnnotations({
        "description": "Hybrid training with function + container customization"
    }),
    
    #  Option 2: Container customization
    WithTrainerImage("python:3.11"),  # Custom base image
    WithTrainerCommand(["python", "-u"]),  # Unbuffered output
    WithTrainerArgs(["-c", "print('🔧 Custom container initialized'); import runpy; runpy.run_module('__main__', run_name='__main__')"])
]

# Submit hybrid training job
job2_name = trainer_client.train(
    trainer=hybrid_trainer,
    runtime=trainer_client.get_runtime("torch-cuda-251"),
    options=hybrid_options
)

print(f" Hybrid TrainJob '{job2_name}' submitted!")

print("\n DEMO 3: Container-Only Training (No Python Function)")
print("=" * 55)

# Container-only approach: No Python function required
container_trainer = CustomTrainer(
    # No func parameter - this is container-only training
    num_nodes=1,
    resources_per_node={
        "cpu": "1",
        "memory": "2Gi"
    }
)

# Container-only options: Pure Option 2
container_options = [
    WithName("mnist-container-only-demo"),
    WithLabels({
        "approach": "container-only",
        "has-function": "false",
        "option": "2"
    }),
    WithAnnotations({
        "description": "Container-only training without Python functions"
    }),
    
    # 🐳 Option 2: Full container control
    WithTrainerImage("python:3.11"),
    WithTrainerCommand(["python", "-c"]),
    WithTrainerArgs([
        "import os, time; "
        "print(' Hello from container-only training!'); "
        "print(f' Node: {os.environ.get(\"HOSTNAME\", \"unknown\")}'); "
        "print(f' Rank: {os.environ.get(\"RANK\", \"0\")}'); "
        "print(' Container-only training completed!'); "
        "time.sleep(30)"  # Keep running for demo
    ])
]

# Submit container-only training job
job3_name = trainer_client.train(
    trainer=container_trainer,
    runtime=trainer_client.get_runtime("torch-cuda-251"),
    options=container_options
)

print(f"Container-only TrainJob '{job3_name}' submitted!")

print("\n ALL DEMOS SUBMITTED SUCCESSFULLY!")
print("=" * 40)
print(f" Jobs created:")
print(f"  1. Traditional: {job1_name}")
print(f"  2. Hybrid: {job2_name}")
print(f"  3. Container-only: {job3_name}")

# Store job names for monitoring
job_names = [job1_name, job2_name, job3_name]

### Monitor Multiple Training Jobs

Monitor all three training approaches and compare their behavior.

In [ ]:
import time

# Get job status
def get_job_status():
    try:
        job = trainer_client.get_job(job_name)
        print(f"Job Status: {job.status}")
        print(f"Created: {job.creation_timestamp}")
        print(f"Runtime: {job.runtime.name}")
        
        if job.steps:
            print("Steps:")
            for step in job.steps:
                print(f"  - {step.name}: {step.status}")
        
        return job.status
    except Exception as e:
        print(f"Error getting job status: {e}")
        return None

# Monitor job status
print("Monitoring job status...")
status = get_job_status()

# Wait for job to start
while status in ["Pending", "Starting", None]:
    print("Waiting for job to start...")
    time.sleep(10)
    status = get_job_status()

print(f"\nJob is now: {status}")


Monitoring job status...
Job Status: Running
Created: 2025-10-05 16:07:43+00:00
Runtime: torch-cuda-251
Steps:
  - node-0: Running
  - node-1: Running

Job is now: Running


In [ ]:
# Get training logs
try:    
    print("\n" + "="*80)
    print("TRAINING LOGS")
    print("="*80)
    
    # Get logs from the training nodes
    logs = trainer_client.get_job_logs(job_name, follow=False)
    
    # Display logs - logs is a generator
    log_count = 0
    for log_line in logs:
        if log_line.strip():
            print(log_line)
            log_count += 1
            # Limit output for demo purposes
            if log_count > 100:
                print("\n... (truncated for demo) ...")
                break
    
    print("\n" + "="*80)
    
except Exception as e:
    print(f"Error getting logs: {e}")
    print("Note: Logs may not be available yet if training is still starting up")


### Cleanup Resources and Summary

Clean up all training jobs and summarize the new SDK features demonstrated.

In [ ]:
def cleanup_trainjob():
    """Clean up the TrainJob using Kubeflow SDK"""
    try:
        trainer_client.delete_job(job_name)
        print(f"✓ TrainJob '{job_name}' deleted successfully")
    except Exception as e:
        print(f"Error deleting TrainJob: {e}")

# Get final job status before cleanup
try:
    final_job = trainer_client.get_job(job_name)
    print(f"Final TrainJob Status:")
    print(f"   Name: {final_job.name}")
    print(f"   Status: {final_job.status}")
    print(f"   Created: {final_job.creation_timestamp}")
    print(f"   Nodes: {final_job.num_nodes}")
    print(f"   Runtime: {final_job.runtime.name}")
    
    if final_job.steps:
        print(f"   Steps:")
        for step in final_job.steps:
            print(f"     - {step.name}: {step.status}")
    
    # Uncomment to automatically cleanup completed jobs
    # if final_job.status in ["Complete", "Failed"]:
    cleanup_trainjob()
            
except Exception as e:
    print(f"Error getting final job status: {e}")



Final TrainJob Status:
   Name: v50712dfc954
   Status: Running
   Created: 2025-10-05 16:16:01+00:00
   Nodes: 2
   Runtime: torch-cuda-251
   Steps:
     - node-0: Running
     - node-1: Running
✓ TrainJob 'v50712dfc954' deleted successfully

DEMO COMPLETE
This notebook demonstrated:
✓ Creating TrainJobs programmatically with Kubeflow SDK
✓ Replicating trl_training.yaml functionality
✓ Distributed TRL training with checkpointing
✓ Real-time monitoring and logging
✓ Advanced progression tracking

To cleanup: Uncomment the cleanup_trainjob() call above
